In [ ]:
import pandas as pd

Read the csv file from this url:
'https://drive.google.com/file/d/1P2EyVCFAPeJcQ_lAN2W-bjwtbizOeFXE/view?usp=drive_link'

Name it `sets_df`

In [ ]:
#Your code here
url = 'https://drive.google.com/file/d/1P2EyVCFAPeJcQ_lAN2W-bjwtbizOeFXE/view?usp=drive_link'
path = 'https://drive.google.com/uc?export=download&id='+url.split('/')[-2]
sets_df = pd.read_csv(path)

#### Exercise 1
How many sets were released in each year?

In [ ]:
#Your code here
sets_df.groupby('year').set_num.count()

year
1954     2
1955     2
1956     1
1957     1
1958     5
        ..
2013    58
2014    71
2015    66
2016    37
2017    34
Name: set_num, Length: 62, dtype: int64

#### Exercise 2
In which five years were the most sets released?

In [ ]:
#Your code here
sets_df.groupby('year').set_num.count().nlargest(5)

year
2014    71
2015    66
2013    58
2010    46
2012    46
Name: set_num, dtype: int64

#### Execise 3
What is the median number of parts and the number of sets in each theme?

In [ ]:
#Your code here
sets_df.groupby('theme_id').agg({'num_parts': 'median', 'set_num': 'count'})

,num_parts,set_num
theme_id,,
1,250.0,14
3,78.0,3
5,353.0,1
6,308.0,1
7,595.0,5
...,...,...
605,1184.0,1
606,615.0,1
610,93.0,1


#### Exercise 4
In which year(s) were the most sets released under a single theme?

In [ ]:
#Your code here
sets_df.groupby(['year', 'theme_id']).set_num.count().sort_values(ascending=False)

year  theme_id
1966  371         7
2014  504         7
      503         6
2001  273         6
2017  484         6
                 ..
2005  397         1
      358         1
      350         1
1990  190         1
2005  33          1
Name: set_num, Length: 664, dtype: int64

#### Load more data
Run the cell below to load more DataFrames for practice on merging.

In [ ]:
def read_data(id):
    return pd.read_csv('https://drive.google.com/uc?export=download&id='+id_dict[id])

id_dict = {
    'sets': '1Dkd09FsucudaWVPkTHRV8Wx_GachJLI1',
    'inventories': '1CoBbtb0XuVcQLk_EqfeQKcfY8EuClOcg',
    'inventory_minifigs': '1e3eogAYpdz0-B_ccdVkfbwOVyNXOckn5',
    'minifigs': '1NOs2Lrxn4Kp-731q8dpBDZkzYpK6WuFm',
    'themes': '1y_RE8f-J91DQBA8qaP-xVlKb_ZZe8ky_',
}

sets_df = read_data('sets')
inventories_df = read_data('inventories')
inventory_minifigs_df = read_data('inventory_minifigs')
minifigs_df = read_data('minifigs')
themes_df = read_data('themes')

Explore all the available DataFrames to get an idea of how they might be merged.

In [ ]:
#Your code here
# e.g.
sets_df.info()
themes_df.info()

left | left_on | right_on | right
---- | ---- | ---- | ----
sets_df | theme_id | id | themes_df
sets_df | set_num | set_num | inventories_df
inventories_df | id | inventory_id | inventory_minifigs_df
inventory_minifigs_df | fig_num | fig_num | minifigs_df

###### **Hint:**

Use something like `.info()` or `.columns` to search for common names.

In [ ]:
themes_df.columns
sets_df.columns
inventories_df.columns
inventory_minifigs_df.columns
minifigs_df.columns

Index(['fig_num', 'name', 'num_parts', 'img_url'], dtype='object')

#### Exercise 5
What is the name of the theme with the most sets associated?

In [ ]:
#Your code here
(
    sets_df
        .merge(themes_df,
               left_on='theme_id',
               right_on='id',
               suffixes=('_set', '_theme')
               )
        .groupby('name_theme')
        ['set_num']
        .count()
        .nlargest(1)
)

name_theme
Star Wars    935
Name: set_num, dtype: int64

#### Exercise 6
What minifigs are in the set 'Traitor Transport (with Cave)'?    

> If two DataFrames have a column of the same name (other than the merging column), you can control the columns' names in the output by using the parameter `suffixes` in the merge.

In [ ]:
#Your code here
(
    sets_df
    .loc[sets_df['name'] == 'Traitor Transport (with Cave)']
    .merge(inventories_df)
    .merge(inventory_minifigs_df, left_on='id', right_on='inventory_id')
    .merge(minifigs_df, on='fig_num', suffixes=('_set', '_fig'))
    [['name_fig']]
)

,name_fig
0,Bat Lord (Fright Knights) - Cape
1,"Knight (Scale Mail on Red) - Black Arms, Blue ..."
2,"Knight (Fright Knights) - Dark Gray Legs, Blac..."


#### Exercise 7
Which set has the has the highest version number?

In [ ]:
#Your code here
sets_df.merge(inventories_df.nlargest(1, 'version'))

,set_num,name,year,theme_id,num_parts,img_url,id,version
0,657-2,1:87 Mercedes Delivery Van,1964,368,1,https://cdn.rebrickable.com/media/sets/657-2.jpg,146112,16


#### Exercise 8
How many minifigs are the set "Mos Eisley Cantina" (2020)?     
Which minifig appears most in that set? (Try to get a picture of it.)

In [ ]:
#Your code here
(
    sets_df.query('(name == "Mos Eisley Cantina")&(year == 2020)')
    .merge(inventories_df)
    .merge(inventory_minifigs_df, left_on='id', right_on='inventory_id')
    .quantity.sum()
)

21

In [ ]:
(
    sets_df.query('(name == "Mos Eisley Cantina")&(year == 2020)')
    .merge(inventories_df)
    .merge(inventory_minifigs_df, left_on='id', right_on='inventory_id')
    .nlargest(1, 'quantity')
    .merge(minifigs_df, on='fig_num')
    [['name_y', 'img_url']]
)

,name_y,img_url
0,Bith Musician,https://cdn.rebrickable.com/media/sets/fig-004...
